# DS-поток, весна 2025
## Задание ADL.4
### Подходы к оптимизации процесса обучения LLMs.

**Правила:**

* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Выполненную работу нужно отправить телеграм-боту `@miptstats_ds24_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. Дождитесь подтверждения от бота, что он принял файл. Если подтверждения нет, то что-то не так. **Работы, присланные иным способом, не принимаются.**
* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb`**.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания получат штраф.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* В каждой задаче не забывайте делать **пояснения и выводы**.
* **Код из рассказанных на занятиях ноутбуков** можно использовать без ограничений.

**Баллы за задание:**

* Реализация &mdash; 80 баллов;
* Сравнение и анализ &mdash; 70 баллов.

In [1]:
import time

import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from collections import defaultdict
import itertools
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Описание

На занятии мы познакомились с различными техниками, которые используются при обучении больших языковых моделей. В этом домашнем задании вам предстоит решить реальную практическую задачу, которая связана с оптимизацией некоторых слоев в трансформере.

Вспомним общую идею техники, которая называется Gradient Checkpointing. Идея заключается в том, чтобы на этапе forward'a не запоминать промежуточные активации, необходимые для backward'a, а вычислять их непосредственно на этапе backward'a. Почему это может быть важно? Оказывается, что активации MLP для больших моделей занимают очень много памяти. Сохранить для backward'а все активации, включая слои внимания, просто невозможно. Возникает вопрос: стоит ли сохранять промежуточные активации MLP или же отдать память под активации attention'a? На практике пересчет активаций для MLP оказывается гораздо быстрее, чем пересчет того же attention'a. В итоге мы можем не сохранять активации MLP, экономить достаточно много памяти, а часть освободившейся памяти отдать под активации attention'a и тем самым даже ускорить обучение!

## Реализация
Сегодня мы не будем работать с полноценным трансформером, а сфокусируемся только на MLP-блоке. Вам предлагается написать модифицированный слой MLP таким образом, чтобы он поддерживал возможность либо сохранять промежуточные активации, либо пересчитывать их на этапе backward'a. Оформить код нужно будет в виде кастомной `torch.autograd.Function`. Хорошая практика заключается в том, чтобы ваш итоговый слой, который наследуется от `torch.nn.Module` просто вызывал функцию с нужными параметрами. Вам нужно **обязательно** ознакомиться с [постом](https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html), в котором вы узнаете, как правильно написать кастомную `torch.autograd.Function` функцию и что это вообще такое.


Вспомним с лекции, как выглядит модифицированный MLP для современных арихетктур.
$$
\text{FFN}_{\text{SwiGLU}}\left(x, W, V, U\right) = \left(\text{Swish}_1\left(xW\right)\otimes xV\right)U = \left(\text{SiLU}\left(xW\right)\otimes xV\right)U,
$$
где
$$
\text{Swish}_1\left(x\right)= \text{SiLU}\left(x\right) = x\sigma\left(x\right)
$$

Такой MLP-блок используется в [LLaMA](https://arxiv.org/abs/2307.09288)-подобных архитектурах. В этом задании будем использовать его. Посмотрим, как выглядит реализация в виде простого `torch.nn.Module`.

In [2]:
class SwigluMLP(nn.Module):
    def __init__(self, hidden_size, intermediate_size):
        super().__init__()
        self.W = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.V = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.U = nn.Linear(intermediate_size, hidden_size, bias=False)
        self.act_fn = nn.SiLU()

    def forward(self, x):
        output = self.U(self.act_fn(self.W(x)) * self.V(x))
        return output

Теперь перейдем к нашей реализации. Обратите внимание, что `MemoryOptimizedSwigluMLPFunction` принимает на вход `checkpoint_level`. Это переменная нужна для реализации следующей логики:
* `checkpoint_level == 0` &mdash; никаких оптимизаций не проводится, промежуточные активации просто сохраняются для переиспользования на этапе backward'a через `ctx.save_for_backward(...)`;
* `checkpoint_level == 1` &mdash; для backward'a сохраняются только вход `x` и матрицы `W, V, U`, а на этапе backward'a нужные активации просто снова пересчитываются.

Вам нужно реализовать методы `forward` и `backward`. Реализация второго потребует от вас посчитать некоторые промежуточные градиенты в матричном виде. Обязательно **выпишите и поясните** получающиеся формулы.

Пусть l = $\frac{\partial L}{\partial F}$

z = $SiLu(xW) \circ xV$

$$
\frac{\partial L}{\partial z} = l \cdot U_t
$$

$$
\frac{\partial z}{\partial xV} = SiLu(xW)
$$


$$
\frac{\partial z}{\partial SiLu(xW)} = xV
$$

Мы помним, что $(x \sigma(x))' = \sigma(x) + x \sigma(x) \cdot(1 - \sigma(x)) = \sigma(x) (1 + x - \sigma(x))$
$$
\frac{\partial SiLu(xW)}{\partial xW} = \sigma(xW) (xW + 1 - \sigma(xW))
$$

$$
\frac{\partial (xW)}{\partial x} = W
$$

$$
\frac{\partial (xV)}{\partial x} = W
$$

Выпишем наконец нужные нам градиенты:

$$
\frac{\partial (L)}{\partial W} = \frac{\partial (L)}{\partial z} \frac{\partial (z)}{\partial SiLu(xW)}\frac{\partial SiLu(xW)}{\partial xW} \cdot x
$$

$$
\frac{\partial (L)}{\partial V} = \frac{\partial (L)}{\partial z} \frac{\partial (z)}{\partial xV} \cdot x
$$


$$
\frac{\partial (L)}{\partial x} = \frac{\partial (L)}{\partial z} \left(\frac{\partial (z)}{\partial SiLu(xW)}\frac{\partial SiLu(xW)}{\partial xW} \cdot W + \frac{\partial (z)}{\partial xV} \cdot V\right)
$$

$$
\frac{\partial (L)}{\partial U} = z^T \cdot l
$$

In [3]:
class MemoryOptimizedSwigluMLPFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, W, V, U, checkpoint_level):
        # x: (batch, seq_len, hidden_dim)
        xW_raw   = torch.matmul(x, W)                   # (batch, seq_len, inter)
        xW_silu  = torch.sigmoid(xW_raw) * xW_raw        # SiLU activation
        xV       = torch.matmul(x, V)                   # (batch, seq_len, inter)
        z        = xW_silu * xV                         # elementwise
        output   = torch.matmul(z, U)                   # (batch, seq_len, out_dim)

        ctx.checkpoint_level = checkpoint_level
        if checkpoint_level == 0:
            ctx.save_for_backward(x, W, V, U, xW_raw, xW_silu, xV)
        else:
            ctx.save_for_backward(x, W, V, U)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        checkpoint_level = ctx.checkpoint_level
        saved = ctx.saved_tensors

        if checkpoint_level == 0:
            x, W, V, U, xW_raw, xW_silu, xV = saved
        else:
            x, W, V, U = saved
            xW_raw   = torch.matmul(x, W)
            xW_silu  = torch.sigmoid(xW_raw) * xW_raw
            xV       = torch.matmul(x, V)

        # 1) вычислим градиент z
        grad_z = grad_output.matmul(U.t())

        # 2) вычислим градиенты по silu(xW) и xV
        grad_xW_silu = grad_z * xV
        grad_xV      = grad_z * xW_silu

        # 3) выпичислим производную silu(xW) по xW
        sigma = torch.sigmoid(xW_raw)
        d_silu = sigma * (1 + xW_raw * (1 - sigma))

        # 4) вычислим  производную L по xW
        grad_xW_raw = grad_xW_silu * d_silu

        # 5) Наконец вычислим необходимые градиенты
        # U: z^T @ grad_output
        z_flat      = (xW_silu * xV).reshape(-1, xW_silu.size(-1))
        go_flat     = grad_output.reshape(-1, grad_output.size(-1))
        grad_U      = z_flat.t().matmul(go_flat)

        # W: x^T @ grad_xW_raw
        dxW_flat    = grad_xW_raw.reshape(-1, grad_xW_raw.size(-1))
        x_flat      = x.reshape(-1, x.size(-1))
        grad_W      = x_flat.t().matmul(dxW_flat)

        # V: x^T @ grad_xV
        dxV_flat    = grad_xV.reshape(-1, grad_xV.size(-1))
        grad_V      = x_flat.t().matmul(dxV_flat)

        # 6) Вычислим градиент по x
        grad_x_from_W = grad_xW_raw.matmul(W.t())
        grad_x_from_V = grad_xV.matmul(V.t())
        grad_x        = grad_x_from_W + grad_x_from_V

        return grad_x, grad_W, grad_V, grad_U, None
        
# Определим новый класс, реализующий оптимизированный MLP-слой
class MemoryOptimizedSwigluMLP(nn.Module):
    def __init__(self, hidden_size, intermediate_size, checkpoint_level):
        super(MemoryOptimizedSwigluMLP, self).__init__()
        self.W = nn.Parameter(torch.empty(hidden_size, intermediate_size))
        self.V = nn.Parameter(torch.empty(hidden_size, intermediate_size))
        self.U = nn.Parameter(torch.empty(intermediate_size, hidden_size))
        self.checkpoint_level = checkpoint_level

    def forward(self, x):
        return MemoryOptimizedSwigluMLPFunction.apply(
            x,
            self.W,
            self.V,
            self.U,
            self.checkpoint_level
        )

Теперь проверим, что реализованный MLP-слой считается верно.

In [4]:
# Зададим параметры
batch_size = 4
seq_len = 256
hidden_dim = 768

dummy_input = torch.randn(batch_size, seq_len, hidden_dim)
# Обратите внимание, что intermediate_size кратно больше hidden_dim
# Это типичное "расширение", характерное для MLP (FFN) слоя в трансформере
swiglu_mlp = SwigluMLP(hidden_dim, hidden_dim * 3)
optimized_swiglu_mlp = MemoryOptimizedSwigluMLP(hidden_dim, hidden_dim * 3, checkpoint_level=1)

# Скопируем параметры, чтобы они были одинаковые
with torch.no_grad():
    optimized_swiglu_mlp.W.data = swiglu_mlp.W.weight.data.t()
    optimized_swiglu_mlp.V.data = swiglu_mlp.V.weight.data.t()
    optimized_swiglu_mlp.U.data = swiglu_mlp.U.weight.data.t()

# Прогоним модель
standard_output = swiglu_mlp(dummy_input)
optimized_output = optimized_swiglu_mlp(dummy_input)

# Проверим выходы слоев на совпадение
assert torch.allclose(standard_output, optimized_output, atol=1e-4)

standard_output.sum().backward()
optimized_output.sum().backward()

# Проверим на совпадение градиенты
assert torch.allclose(swiglu_mlp.U.weight.grad, optimized_swiglu_mlp.U.grad.t(), atol=1e-4)
assert torch.allclose(swiglu_mlp.V.weight.grad, optimized_swiglu_mlp.V.grad.t(), atol=1e-4)
assert torch.allclose(swiglu_mlp.W.weight.grad, optimized_swiglu_mlp.W.grad.t(), atol=1e-4)
print("ALL ASSERTS PASSED")

ALL ASSERTS PASSED


## Сравнение и анализ

Сравните время исполнение forward/backward и объем потребляемой памяти в зависимости от значения `checkpoint_level`. Проведите эксперименты для разных значений `batch_size`, `seq_len`, `hidden_dim`. Сделайте запуски в нескольких сетапах. Попробуйте достаточно большие `seq_len=1024` и `hidden_dim=4096`, а также `num_layers=5`. Сделайте выводы.

Некоторые советы:
* Для отслеживания потребляемой памяти можете воспользоваться `torch.cuda.max_memory_allocated()`. Желательно после каждого шага очищать статистику, используя `torch.cuda.reset_peak_memory_stats()`. Более подробно рекомендуется почитать документацию по [ссылке](https://pytorch.org/docs/stable/notes/cuda.html#cuda-memory-management).
* Для подсчета времени можно воспользоваться простым ` time.perf_counter()` или же `time.time()`. Однако с подсчетом времени для GPU-операций все немного хитрее. В PyTorch и других библиотеках, работающих с GPU, операции выполняются асинхронно по отношению к коду, исполняемому на CPU. Такой подход позволяет CPU продолжать работу, не ожидая окончания каждой операции на GPU, что способствует повышению общей производительности за счёт параллельной работы CPU и GPU. Что это значит на практике? Вы можете получить завышенные результаты своих измерений, так как замер времени может завершиться до того, как GPU в действительности закончит выполнение операций.
Рассмотрим пример кода:
```
a = torch.rand(10000, 10000, device="cuda")
start_time = time.time()
b = a @ a
elapsed_time = time.time() - start_time
```
В этом примере, после запуска операции умножения матриц `a @ a`, мы немедленно измеряем время выполнения функции, не дожидаясь её фактического завершения на GPU.
Для получения точных измерений времени выполнения операций на GPU необходимо использовать синхронизацию. В PyTorch это можно сделать с помощью функции `torch.cuda.synchronize()`, которая блокирует выполнение кода на CPU до тех пор, пока все запланированные задачи на соответствующем GPU не будут завершены. \
Пример более грамотного кода:
```
a = torch.rand(10000, 10000, device='cuda')
torch.cuda.synchronize() # ждем завершения всех предыдущих операций на GPU
start_time = time.time()
b = a @ a
torch.cuda.synchronize() # cнова синхронизируемся, чтобы убедиться, что операция завершена
elapsed_time = time.time() - start_time
```
* Для подсчета статистики следует сделать несколько проходов forward/backward для одной модели, а полученные результаты просто усреднить. Для более стабильных результатов выполните также разогрев, то есть некоторое количество прогонов модели перед основным измерением. Это важно, т.к. на результаты измерений могут повлиять дополнительные задержки, связанные с инициализацией и загрузкой ресурсов, температурой GPU, а также различные кэши.
* Представьте результаты и выводы в информативном виде, хорошо подойдет какая-нибудь табличка. Затраты по памяти лучше всего указать в Гб, а время исполнения в секундах.

In [5]:
# используем несколько слоев, чтобы увидеть выигрыш по памяти
# в случае chechkpoint_level == 0 для вычислений очередного слоя будет использована та память, 
# что осталась для предыдущего
# если же checkpoint_level == 1, то придется хранить активации для всех слоев

class TestTransformer(nn.Module):
    def __init__(self, num_layers, hidden_size, intermediate_size, checkpoint_level):
        super(TestTransformer, self).__init__()
        self.layers = nn.ModuleList([
            MemoryOptimizedSwigluMLP(hidden_size, intermediate_size, checkpoint_level)
            for _ in range(num_layers)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

Реализация функции для бенчмарка.

In [6]:
def benchmark_transformer(model, batch_size, seq_len, hidden_dim, num_warmup_steps, num_steps):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    statistics = defaultdict(list)
    
    # Генерация входных данных
    input_tensor = torch.randn(batch_size, seq_len, hidden_dim, device=device, requires_grad=True)
    model.to(device)

    # Warmup GPU чтобы избежать заниженного перфоманса
    for _ in range(num_warmup_steps):
        output = model(input_tensor)
        output.sum().backward()
        torch.cuda.synchronize()

    # Основное измерение
    for _ in range(num_steps):
        torch.cuda.reset_peak_memory_stats()
        
        torch.cuda.synchronize()
        start_time_forward = time.time()
        
        output = model(input_tensor)
        
        torch.cuda.synchronize()
        end_time_forward = time.time()

        torch.cuda.synchronize()
        start_time_backward = time.time()
        
        output.sum().backward()

        torch.cuda.synchronize()
        end_time_backward = time.time()

        allocated_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)
        time_consumed_forward = end_time_forward - start_time_forward
        time_consumed_backward = end_time_backward - start_time_backward

        statistics["forward_time, s"].append(round(time_consumed_forward, 3))
        statistics["backward_time, s"].append(round(time_consumed_backward, 3))
        statistics["peak_memory, Gb"].append(round(allocated_memory, 3))
    
    statistics_df = pd.DataFrame(statistics).mean(axis=0)
    return statistics_df

In [7]:
def run_experiments():
    experiment_configs = [
    # seq_len = 128, hidden_dim = 512, num_layers = 2
    {"batch_size": 16,  "seq_len": 128,  "hidden_dim": 512,  "intermediate_size": 2048,  "num_layers": 2, "checkpoint_level": 0},
    {"batch_size": 16,  "seq_len": 128,  "hidden_dim": 512,  "intermediate_size": 2048,  "num_layers": 2, "checkpoint_level": 1},
    {"batch_size": 32,  "seq_len": 128,  "hidden_dim": 512,  "intermediate_size": 2048,  "num_layers": 2, "checkpoint_level": 0},
    {"batch_size": 32,  "seq_len": 128,  "hidden_dim": 512,  "intermediate_size": 2048,  "num_layers": 2, "checkpoint_level": 1},

    # seq_len = 512, hidden_dim = 1024, num_layers = 3
    {"batch_size": 8,  "seq_len": 512,  "hidden_dim": 1024, "intermediate_size": 4096,  "num_layers": 3, "checkpoint_level": 0},
    {"batch_size": 8,  "seq_len": 512,  "hidden_dim": 1024, "intermediate_size": 4096,  "num_layers": 3, "checkpoint_level": 1},
    {"batch_size": 16,  "seq_len": 512,  "hidden_dim": 1024, "intermediate_size": 4096,  "num_layers": 3, "checkpoint_level": 0},
    {"batch_size": 16,  "seq_len": 512,  "hidden_dim": 1024, "intermediate_size": 4096,  "num_layers": 3, "checkpoint_level": 1},

    # seq_len = 1024, hidden_dim = 4096, num_layers = 5
    {"batch_size": 4,  "seq_len": 1024, "hidden_dim": 4096, "intermediate_size": 16384, "num_layers": 5, "checkpoint_level": 0},
    {"batch_size": 4,  "seq_len": 1024, "hidden_dim": 4096, "intermediate_size": 16384, "num_layers": 5, "checkpoint_level": 1},
    ]
    
    all_results = []

    for config in tqdm(experiment_configs):
        print(f"Running: {config}")
        model = TestTransformer(
            num_layers=config["num_layers"],
            hidden_size=config["hidden_dim"],
            intermediate_size=config["intermediate_size"],
            checkpoint_level=config["checkpoint_level"]
        )
        stats = benchmark_transformer(
            model=model,
            batch_size=config["batch_size"],
            seq_len=config["seq_len"],
            hidden_dim=config["hidden_dim"],
            num_warmup_steps=2,
            num_steps=15,
        )

        stats = stats.to_dict()
        stats.update(config)
        all_results.append(stats)

    return pd.DataFrame(all_results)

In [8]:
run_experiments()

  0%|          | 0/10 [00:00<?, ?it/s]

Running: {'batch_size': 16, 'seq_len': 128, 'hidden_dim': 512, 'intermediate_size': 2048, 'num_layers': 2, 'checkpoint_level': 0}
Running: {'batch_size': 16, 'seq_len': 128, 'hidden_dim': 512, 'intermediate_size': 2048, 'num_layers': 2, 'checkpoint_level': 1}
Running: {'batch_size': 32, 'seq_len': 128, 'hidden_dim': 512, 'intermediate_size': 2048, 'num_layers': 2, 'checkpoint_level': 0}
Running: {'batch_size': 32, 'seq_len': 128, 'hidden_dim': 512, 'intermediate_size': 2048, 'num_layers': 2, 'checkpoint_level': 1}
Running: {'batch_size': 8, 'seq_len': 512, 'hidden_dim': 1024, 'intermediate_size': 4096, 'num_layers': 3, 'checkpoint_level': 0}
Running: {'batch_size': 8, 'seq_len': 512, 'hidden_dim': 1024, 'intermediate_size': 4096, 'num_layers': 3, 'checkpoint_level': 1}
Running: {'batch_size': 16, 'seq_len': 512, 'hidden_dim': 1024, 'intermediate_size': 4096, 'num_layers': 3, 'checkpoint_level': 0}
Running: {'batch_size': 16, 'seq_len': 512, 'hidden_dim': 1024, 'intermediate_size': 4096

,"forward_time, s","backward_time, s","peak_memory, Gb",batch_size,seq_len,hidden_dim,intermediate_size,num_layers,checkpoint_level
0,0.004267,0.009267,0.305,16,128,512,2048,2,0
1,0.004000,0.012000,0.258,16,128,512,2048,2,1
2,0.008000,0.018000,0.535,32,128,512,2048,2,0
3,0.008000,0.023000,0.442,32,128,512,2048,2,1
4,0.042133,0.092400,1.469,8,512,1024,4096,3,0
5,0.042200,0.118533,1.094,8,512,1024,4096,3,1
6,0.080267,0.180000,2.594,16,512,1024,4096,3,0
7,0.080267,0.232867,1.844,16,512,1024,4096,3,1
8,0.993467,2.016467,14.391,4,1024,4096,16384,5,0
9,0.993933,2.667533,11.391,4,1024,4096,16384,5,1


**Вывод:**

Мы видим, что честный обсчет активаций увеличивает премя работы backward pass примерно на 20%, но при этом экономится порядка 15-30% процентов от затраченной видеопамяти. Причем экономия спадает с увеличением значений парметров.

Таким образом, мы действительно наблюдаем выигрыш в видеопамяти, но нужно учитывать, что он обходится нам увеличением времени обучения.